# Deutsch-Jozsa

## Implementando f como um circuito de forma aleatória

Para a função constante:
dj deve atuar como um circuito que recebe os imputs dos n qubits e trasnforma o qubit n+1 (auxiliar) em (q_aux + f(x))
então, f(x) não pode depender do imput, logo, o código começa com 50% de chance de ser constante, e se isso for o caso, tem 50% do qubit auxiliar continuar igual ou flipar com a porta X


Para a fumção balanceada: escolha-se metade dos estados possíveis em "on_state", dessa forma, Uf age apenas nestes estados da seguinte forma:
caso um estado marcado passe pelo circuito, este vai fazer com que o qubit auxiliar flipe, ficando |q_aux + 1>, significando que f(|s>) = 1 para o estado |s> que está marcado.

para que isso aconteca, o código verifica os estados marcados e flipa q_aux com o multi-control-X-gate (mcx), onde caso o estado seja |111...1> o q_aux é flipado. Dessa forma, é preciso transformar o estado s em |111...1> com a porta X nos qubits especificos ligado ao bit do bitstring s, onde após o mcx, é adicionado mais um X para voltar ao estado anterior |s>.

lembrando que as portas operam em todos os estados, mas pela configuração das portas, apenas os estados marcados farão com que f(x) = 1. fazendo isso, não significa que os estados passarão um por um e mudar f(x) no resultado final, o que acontece é apenas a implementação do funcionamento de f(x) com o circuito.


In [2]:
from qiskit import QuantumCircuit
import numpy as np

def dj_function(num_qubits):
    """
    Create a random Deutsch-Jozsa function.
    """

    qc = QuantumCircuit(num_qubits + 1)
    if np.random.randint(0, 2):
        # Flip output qubit with 50% chance
        qc.x(num_qubits)
    if np.random.randint(0, 2):
        # return constant circuit with 50% chance
        return qc

    # next, choose half the possible input states
    on_states = np.random.choice(
        range(2**num_qubits),  # numbers to sample from
        2**num_qubits // 2,  # number of samples
        replace=False,  # makes sure states are only sampled once
    )

    def add_cx(qc, bit_string):
        for qubit, bit in enumerate(reversed(bit_string)):
            if bit == "1":
                qc.x(qubit)
        return qc

    for state in on_states:
        qc.barrier()  # Barriers are added to help visualize how the functions are created. They can safely be removed.
        qc = add_cx(qc, f"{state:0b}")
        qc.mcx(list(range(num_qubits)), num_qubits)
        qc = add_cx(qc, f"{state:0b}")

    qc.barrier()

    return qc

In [3]:
display(dj_function(3).draw())

q_0: ─────
          
q_1: ─────
          
q_2: ─────
     ┌───┐
q_3: ┤ X ├
     └───┘

In [4]:
def compile_circuit(function: QuantumCircuit):
    """
    Compiles a circuit for use in the Deutsch-Jozsa algorithm.
    """
    n = function.num_qubits - 1
    qc = QuantumCircuit(n + 1, n)
    qc.x(n)
    qc.h(range(n + 1))
    qc.compose(function, inplace=True)
    qc.h(range(n))
    qc.measure(range(n), range(n))
    return qc

In [5]:
from qiskit_aer import AerSimulator

def dj_algorithm(function: QuantumCircuit):
    """
    Determine if a Deutsch-Jozsa function is constant or balanced.
    """
    qc = compile_circuit(function)

    result = AerSimulator().run(qc, shots=1, memory=True).result()
    measurements = result.get_memory()
    if "1" in measurements[0]:
        return "balanced"
    return "constant"

In [6]:
f = dj_function(4)
display(f.draw())
display(dj_algorithm(f))

q_0: 
     
q_1: 
     
q_2: 
     
q_3: 
     
q_4:

'constant'

# Berstein-Vazirani problem

Definindo uma função que implementa uma query gate par ao problema de B.V. dado uma string binária "s"

para que f(x) = s * x seja implementado, os bits "1" do bitsring s, fará com que os bits de x naquela posição permanecam no resultado.
ex: s = 1001, então s*x = x_3 + x_0 (mod 2), logo para que f(x) = q_3 + q_0 (mod 2), implementa-se uma X gate em q_0 e q_3.


In [14]:
def bv_function(s):
    qc = QuantumCircuit(len(s) + 1)
    for index, bit in enumerate(reversed(s)):
        if bit == "1":
            qc.cx(index, len(s))
    return qc

In [ ]:
def bv_algorithm(function: QuantumCircuit):
    qc = compile_circuit(function)
    result = AerSimulator().run(qc, shots=1, memory=True).result()
    return result.get_memory()[0]

display(bv_function("1001011101").draw())

display(bv_algorithm(bv_function("1001011101")))

q_0: ──■───────────────────────────
        │                           
 q_1: ──┼───────────────────────────
        │                           
 q_2: ──┼────■──────────────────────
        │    │                      
 q_3: ──┼────┼────■─────────────────
        │    │    │                 
 q_4: ──┼────┼────┼────■────────────
        │    │    │    │            
 q_5: ──┼────┼────┼────┼────────────
        │    │    │    │            
 q_6: ──┼────┼────┼────┼────■───────
        │    │    │    │    │       
 q_7: ──┼────┼────┼────┼────┼───────
        │    │    │    │    │       
 q_8: ──┼────┼────┼────┼────┼───────
        │    │    │    │    │       
 q_9: ──┼────┼────┼────┼────┼────■──
      ┌─┴─┐┌─┴─┐┌─┴─┐┌─┴─┐┌─┴─┐┌─┴─┐
q_10: ┤ X ├┤ X ├┤ X ├┤ X ├┤ X ├┤ X ├
      └───┘└───┘└───┘└───┘└───┘└───┘

'1001011101'

: 